In [4]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

def ollama_chat_client():
    return OpenAIChatClient(
        api_key="ollama",
        model_id="gemma3:1b",
        base_url="http://localhost:11434/v1"  # la URL de tu servidor Ollama
    )

agent = ChatAgent(
    chat_client=ollama_chat_client(),
    instructions="You are a helpful assistant.",
    name="LocalOllamaAgent"
)


In [7]:
agent_response = await agent.run("Tell me a joke about a pirate.")
agent_response.text


'Why did the pirate cross the playground? \n\nTo get to the pirate cove! \n\n---\n\nWould you like another one? 😊'

In [8]:
# requirements: pip install agent-framework --pre
import asyncio
from agent_framework import (
    WorkflowBuilder,
    WorkflowContext,
    Executor,
    handler,
    RequestInfoEvent,
    WorkflowOutputEvent,
)

from dataclasses import dataclass

In [24]:
from agent_framework import Executor, handler, WorkflowContext, WorkflowBuilder

@dataclass
class UploadExcelRequest:
    prompt: str = "Please upload your Excel file containing the data."

class RequestExcelExecutor(Executor):
    @handler
    async def request_excel(self, _: None, ctx: WorkflowContext[str]):
        await ctx.request_info(request_data=UploadExcelRequest())

    @handler
    async def handle_upload(self, message: str, ctx: WorkflowContext[str]):
        # Aquí guardás/lees el Excel si querés
        await ctx.send_message(message)

class ValidateExcelExecutor(Executor):
    @handler
    async def validate_excel(self, excel_path: str, ctx: WorkflowContext[str]):
        # leés el excel de excel_path, validás...
        # Si está ok:
        await ctx.send_message("ok")
        # O si querés podrías también ctx.yield_output("Excel OK")

class FinalExecutor(Executor):
    @handler
    async def finish(self, msg: str, ctx: WorkflowContext[None]):
        print("Flujo finalizado, mensaje:", msg)


In [25]:
req = RequestExcelExecutor(id="request_excel")
val = ValidateExcelExecutor(id="validate_excel")
fin = FinalExecutor(id="final")

builder = WorkflowBuilder()
builder.set_start_executor(req)
builder.add_edge(req, val)       # usa la misma instancia `req`
builder.add_edge(val, fin)
workflow = builder.build()


In [26]:
async def run_workflow():
    pending: dict | None = None
    completed = False
    while not completed:
        stream = workflow.send_responses_streaming(pending) if pending else workflow.run_stream("start")
        events = [event async for event in stream]
        pending = None

        for ev in events:
            if isinstance(ev, RequestInfoEvent):
                # Mostrá prompt al usuario, pedí subida de archivo, etc.
                print(ev.data.prompt)
                # aquí tu lógica para recibir el archivo, guardarlo, devolver la ruta
                file_path = input("Ingresa la ruta del archivo Excel: ")
                pending = { ev.request_id: file_path }
            elif isinstance(ev, WorkflowOutputEvent):
                print("OUTPUT:", ev.data)
                completed = True

In [27]:
await run_workflow()


Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo finalizado, mensaje: ok
Flujo fina

CancelledError: 